In [ ]:
%pip install selenium bs4
# docs to use:
# https://www.selenium.dev/documentation/
# https://beautiful-soup-4.readthedocs.io/en/latest/


In [ ]:
# function to extract api information
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By


def getApiRefDict(referenceUrl):
    browser.get(referenceUrl)
    soup = BeautifulSoup(browser.page_source)

    # the dict to be returned, layered like the website
    # e.g. the soup element of "find-funding-accounts" can be obtained by:
    #   apiRefDict['collect']['funding']['find-funding-accounts']
    apiRefDict = {}
    
    # turns out the h1 heading elements are used for the categories 
    # like "Collect", "Convert", "Manage" etc.
    subPackageHeadings0 = soup.find_all("h1")
    # To drop the first useless h1 heading which is "API REFERENCE"
    subPackageHeadings0.pop(0)
    for subPackageHeading0 in subPackageHeadings0:
        # e.g. "collect"
        subPackageName0 = subPackageHeading0["id"]
        
        subDict1 = {}
        subPackage1 = subPackageHeading0.next_sibling
        while(subPackage1):
            subPackageHeading1 = subPackage1.contents[0]
            # e.g. "funding"
            subPackageName1 = subPackageHeading1["id"]
            
            endpointDict = {}
            endpoint = subPackageHeading1.next_sibling
            while(endpoint):
                # e.g. "find-funding-accounts"
                id = endpoint["id"]
                
                detailedEndpoint = getDetailedEndpointData(endpoint)
                
                endpointDict[id] = detailedEndpoint
                endpoint = endpoint.next_sibling
                
            subDict1[subPackageName1] = endpointDict
            subPackage1 = subPackage1.next_sibling
            
        apiRefDict[subPackageName0] = subDict1
    return apiRefDict


def getDetailedEndpointData(endpoint):
    # e.g. "find-funding-accounts"
    id = endpoint["id"]
    
    # goto the page with the specific endpoint and 
    # press all the buttons in parameters section to
    # expand and reveal all info, then 
    # extract and store the soup element of the endpoint to dict
    referenceUrlEndpointId = referenceUrl + "#" + id
    browser.get(referenceUrlEndpointId)
    browserEndpoint = browser.find_element(By.ID, id)
    browserParams = browserEndpoint.find_element(By.CLASS_NAME, "api-endpoint__parameters")
    for button in browserParams.find_elements(By.TAG_NAME, "button"):
        button.click()
    detailedSoup = BeautifulSoup(browser.page_source)
    
    endpointData = detailedSoup.find(id=id)
    return endpointData

In [ ]:
# function to create folder structure
import os
import shutil


def createFolderStructure(apiRefDict, packageName):
    cwd = os.getcwd()
    packagePath = os.path.join(cwd, packageName)
    # delete and remake the top folder if it exists
    shutil.rmtree(packagePath, ignore_errors=True)
    os.mkdir(packagePath)
    for subPackageName1 in apiRefDict:
        subPackagePath1 = os.path.join(packagePath, subPackageName1)
        os.mkdir(subPackagePath1)
        for subPackageName2 in apiRefDict[subPackageName1]:
            subPackagePath2 = os.path.join(subPackagePath1, subPackageName2)
            os.mkdir(subPackagePath2)
            subPackageRequestsPath2 = os.path.join(subPackagePath2, "requests")
            os.mkdir(subPackageRequestsPath2)
    return


In [ ]:
# functions to create services
import os
import re


def createServices(apiRefDict, packageName):
    for subPackageName1 in apiRefDict:
        for subPackageName2 in apiRefDict[subPackageName1]:
            packageNames = [packageName, subPackageName1, subPackageName2]
            endpoints = apiRefDict[subPackageName1][subPackageName2].values()
            createService(endpoints, packageNames)
    return


def createService(endpoints, packageNames):
    # the starting part of the file
    serviceContent = writeServiceHeadStr(packageNames)
    
    # figure out the parts within the class
    for endpoint in endpoints:
        funcStr = writeServiceFuncStr(endpoint, packageNames)
        serviceContent = serviceContent + funcStr
        
    serviceContent = serviceContent + "\n}"
    
    # get the path of the file
    servicePath = os.getcwd()
    for name in packageNames:
        servicePath = os.path.join(servicePath, name)
    serviceName = packageNames[2].title() + "Service.java"
    servicePath = os.path.join(servicePath, serviceName)
    # write content to the file
    f = open(servicePath, "w")
    f.write(serviceContent)
    f.close()
    return


def writeServiceHeadStr(packageNames):
    headStr = f"""
package com.mynt.banking.currency_cloud.{packageNames[1]}.{packageNames[2]};

import com.fasterxml.jackson.databind.JsonNode;
import com.fasterxml.jackson.databind.ObjectMapper;
import com.mynt.banking.currency_cloud.manage.authenticate.AuthenticationService;
import com.mynt.banking.currency_cloud.{packageNames[1]}.{packageNames[2]}.requests.*;
import com.mynt.banking.util.HashMapToQuiryPrams;
import lombok.RequiredArgsConstructor;
import org.springframework.http.ResponseEntity;
import org.springframework.stereotype.Service;
import org.springframework.web.reactive.function.client.WebClient;
import reactor.core.publisher.Mono;

import java.util.HashMap;

@RequiredArgsConstructor
@Service
public class {packageNames[2].title()}Service {{

    private final AuthenticationService authenticationService;
    private final WebClient webClient;
    """
    return headStr



def writeServiceFuncStr(endpoint, packageNames):
    id = endpoint['id']
    capName = id.replace('-',' ').title().replace(' ','')
    camelName = capName[0].lower() + capName[1:]
    
    parameters = endpoint.find("div", class_="api-endpoint__parameters")
    method = parameters.find("div", method=["POST", "GET"])['method']
    url = parameters.find("div", method=["POST", "GET"]).contents[1].string
    needsAuth = parameters.find("span", string=re.compile("X-Auth-Token"))
    path = parameters.find("h5", string="Path")
    
    # if there is a path list, modify the url to include the variables
    if path:
        for button in path.next_sibling.find_all("button"):
            buttonStr = button.contents[1].get_text().replace("*","")
            url = url.replace(
                "{" + buttonStr + "}", 
                "\" + " + buttonStr + " + \""
                )

    # head of function
    funcHeadStr = f"""
    public Mono<ResponseEntity<JsonNode>> {camelName}(
    """
    
    if path:
        for button in path.next_sibling.find_all("button"):
            buttonStr = button.contents[1].get_text().replace("*","")
            funcHeadStr = funcHeadStr + f"\t\tString {buttonStr},"
            
    funcHeadStr = funcHeadStr + f"""
            {capName}Request request
    ) {{"""
    
    # body of function
    if method == "GET":
        funcBodyStr = f"""
        ObjectMapper objectMapper = new ObjectMapper();
        HashMap<String, Object> prams = objectMapper.convertValue(request, HashMap.class);
        String url = "{url}" + HashMapToQuiryPrams.hashMapToString(prams);

        return webClient
                .get()
                .uri(url)
                {"" if needsAuth else "//"}.header("X-Auth-Token", authenticationService.getAuthToken())
                .exchangeToMono(response -> response.toEntity(JsonNode.class))
                .flatMap(response -> Mono.just(response));

    }}
        """
    else:
        funcBodyStr = f"""
        return webClient
                .post()
                .uri("{url}")
                {"" if needsAuth else "//"}.header("X-Auth-Token", authenticationService.getAuthToken())
                .bodyValue(requestBody)
                .exchangeToMono(response -> response.toEntity(JsonNode.class))
                .flatMap(Mono::just);
        """
    
    funcStr = funcHeadStr + funcBodyStr
    return funcStr


In [ ]:
# functions to create controllers
import os


def createControllers(apiRefDict, packageName):
    for subPackageName1 in apiRefDict:
        for subPackageName2 in apiRefDict[subPackageName1]:
            packageNames = [packageName, subPackageName1, subPackageName2]
            endpoints = apiRefDict[subPackageName1][subPackageName2].values()
            createController(endpoints, packageNames)
    return


def createController(endpoints, packageNames):
    # the starting part of the file
    controllerContent = writeControllerHeadStr(packageNames)
    
    # figure out the parts within the class
    for endpoint in endpoints:
        funcStr = writeControllerFuncStr(endpoint, packageNames)
        controllerContent = controllerContent + funcStr
        
    controllerContent = controllerContent + "\n}"
    
    # get the path of the file
    filePath = os.getcwd()
    for name in packageNames:
        filePath = os.path.join(filePath, name)
    fileName = packageNames[2].title() + "Controller.java"
    filePath = os.path.join(filePath, fileName)
    # write to file
    f = open(filePath, "w")
    f.write(controllerContent)
    f.close()
    return


def writeControllerHeadStr(packageNames):
    headStr = f"""
package com.mynt.banking.currency_cloud.{packageNames[1]}.{packageNames[2]};

import com.fasterxml.jackson.databind.JsonNode;
import com.mynt.banking.currency_cloud.{packageNames[1]}.{packageNames[2]}.requests.*;
import lombok.RequiredArgsConstructor;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;
import reactor.core.publisher.Mono;

@RestController
@RequestMapping("/api/v1/currency-cloud")
@RequiredArgsConstructor
public class {packageNames[2].title()}Controller {{
    
    private final {packageNames[2].title()}Service {packageNames[2]}Service;
    """
    return headStr


def writeControllerFuncStr(endpoint, packageNames):
    id = endpoint['id']
    capName = id.replace('-',' ').title().replace(' ','')
    camelName = capName[0].lower() + capName[1:]
    
    parameters = endpoint.find("div", class_="api-endpoint__parameters")
    url = parameters.find("div", method=["POST","GET"]).contents[1].string
    path = parameters.find("h5", string="Path")
    
    funcHeadStr = f"""
    @PostMapping("{url}")
    public Mono<ResponseEntity<JsonNode>> {camelName}("""
    
    if path:
        for button in path.next_sibling.find_all("button"):
            buttonStr = button.contents[1].get_text().replace("*","")
            buttonDescription = button.next_sibling.get_text().replace("\"", "\\\"")
            funcHeadStr = funcHeadStr + f"""
            @Schema(description = "{buttonDescription}")
            @PathVariable(name = "{buttonStr}") String {buttonStr},"""
            
    funcHeadStr = funcHeadStr + f"""
            @RequestBody {capName}Request request
    ) {{"""
    
    funcBodyStr = f"""
        return {packageNames[2]}Service.{camelName}("""
        
    if path:
        for button in path.next_sibling.find_all("button"):
            buttonStr = button.contents[1].get_text().replace("*","")
            buttonDescription = button.next_sibling.get_text().replace("\"", "\\\"")
            funcBodyStr = funcBodyStr + f"""
            {buttonStr},"""
        
    funcBodyStr = funcBodyStr + f"""
            request
        );
    }}
    """
    funcStr = funcHeadStr + funcBodyStr
    return funcStr


In [ ]:
# functions to create requests
import os


def createRequests(apiRefDict, packageName):
    for subPackageName1 in apiRefDict:
        for subPackageName2 in apiRefDict[subPackageName1]:
            packageNames = [packageName, subPackageName1, subPackageName2]
            for endpoint in apiRefDict[subPackageName1][subPackageName2].values():
                createRequest(endpoint, packageNames)
    return


def createRequest(endpoint, packageNames):
    # the starting part of the file
    requestContent = writeRequestHeadStr(endpoint, packageNames)
    
    # figure out the parts within the class
    queryFormSoup = endpoint.find("h5", string=["Query","Form data"])
    if queryFormSoup:
        for queryFormButton in queryFormSoup.next_sibling.find_all("button"):
            funcStr = writeRequestFuncStr(queryFormButton)
            requestContent = requestContent + funcStr
    requestContent = requestContent + "\n}"
    
    # get the path of the file
    filePath = os.getcwd()
    for name in packageNames:
        filePath = os.path.join(filePath, name)
    filePath = os.path.join(filePath, "requests")
    fileName = endpoint['id'].replace('-', ' ').title().replace(' ', '') + "Request.java"
    filePath = os.path.join(filePath, fileName)
    # write content to the file
    f = open(filePath, "w")
    f.write(requestContent)
    f.close()
    return


def writeRequestHeadStr(endpoint, packageNames):
    capName = endpoint['id'].replace('-',' ').title().replace(' ','')
    description = endpoint.contents[2].find("p").get_text().replace("\"", "\\\"")
    
    initialStr = f"""
package com.mynt.banking.currency_cloud.{packageNames[1]}.{packageNames[2]}.requests;

import com.fasterxml.jackson.annotation.JsonInclude;
import com.fasterxml.jackson.annotation.JsonProperty;
import io.swagger.v3.oas.annotations.media.Schema;
import jakarta.validation.constraints.NotNull;
import jakarta.validation.constraints.Size;
import lombok.Builder;
import lombok.Data;

import java.util.Date;

@Data
@Builder
@JsonInclude(JsonInclude.Include.NON_EMPTY)
@Schema(description = "{description}")
public class {capName}Request {{
    """
    
    return initialStr


def writeRequestFuncStr(button):
    name = button.find("span").get_text()
    isMandatory = name[-1]=='*'
    if isMandatory:
        name = name[:-1]
    description = button.next_sibling.find("p").string
    capName = name.replace('_',' ').title().replace(' ','')
    camelName = capName[0].lower() + capName[1:]
    
    funcStr = f"""
    {"" if isMandatory else "//"}@NotNull
    @JsonProperty("{name}")
    @Size(max = 255)
    @Schema(description = "{description}",
            example = " ")
    @Builder.Default
    private String {camelName} = "";
    """
    
    return funcStr

In [ ]:
# initialize browser
from selenium import webdriver
import time

# the link to api-reference, needs a "/" at the end for later use # to goto sections
referenceUrl = "https://developer.currencycloud.com/api-reference/"
# start a browser
browser=webdriver.Firefox()
# get the browser to load the api reference page
browser.get(referenceUrl)
time.sleep(2)
# reject the cookies
browser.find_element(By.ID, "CookieReportsPanel").find_element(By.LINK_TEXT, "Reject all").click()
time.sleep(2)

    


In [ ]:
# get dict of api

# this is to define the name of the top folder
packageName = "currency_cloud"
# get the dict which contains the detailed soup element of endpoints
# with the structure like the folders
apiRefDict = getApiRefDict(referenceUrl)


In [ ]:
# call functions to create folders and files
createFolderStructure(apiRefDict, packageName)
createServices(apiRefDict, packageName)
createControllers(apiRefDict, packageName)
createRequests(apiRefDict, packageName)